<a href="https://colab.research.google.com/github/Ahmad83-pixel/Yolo11/blob/main/Multi_Objects_Tracking_using_Multi_Cameras_using_hyprid_Yolo11_model_ipynd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**3- Cameras Access to record / real time process**

1- Mount google drive from objecttrackign.ahmad@gmail.com

In [1]:
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [104]:
from IPython.display import display, HTML

display(HTML("""
<div style="margin: 20px; text-align: center;">
    <h2>Multi Objects Tracking using Multi Cameras</h2>
    <div id="camera-container" style="display: flex; justify-content: center; gap: 20px; flex-wrap: wrap;">
        <!-- Camera 1 -->
        <div id="cam1" style="width: 320px; height: 240px; border: 2px solid #ddd; background: #000; overflow: hidden; position: relative;">
            <video id="video1" autoplay playsinline muted style="width: 100%; height: 100%; object-fit: cover;"></video>
            <div class="camera-label">Camera 1</div>
        </div>
        <!-- Camera 2 -->
        <div id="cam2" style="width: 320px; height: 240px; border: 2px solid #ddd; background: #000; overflow: hidden; position: relative;">
            <video id="video2" autoplay playsinline muted style="width: 100%; height: 100%; object-fit: cover;"></video>
            <div class="camera-label">Camera 2</div>
        </div>

    </div>
    <p id="status" style="color: #666; margin-top: 20px;">Initializing cameras...</p>
    <button id="retry-btn" style="padding: 10px 20px; background: #2196F3; color: white; border: none; border-radius: 5px; margin-top: 15px;">Reinitialize Cameras</button>
</div>

<style>
.camera-label {
    position: absolute;
    top: 10px;
    left: 10px;
    background: rgba(0,0,0,0.7);
    color: white;
    padding: 2px 8px;
    border-radius: 4px;
}
.camera-error {
    width:100%;
    height:100%;
    display:flex;
    align-items:center;
    justify-content:center;
    background:#222;
    color:white;
}
</style>

<script>
const cameraSystem = {
    streams: [],
    devices: []
};

async function setupCameras() {
    const status = document.getElementById('status');
    status.textContent = "⌛ Detecting cameras...";
    status.style.color = "#666";

    // Clear previous streams
    cameraSystem.streams.forEach(stream => {
        stream.getTracks().forEach(track => track.stop());
    });
    cameraSystem.streams = [];

    try {
        // First get basic access
        await navigator.mediaDevices.getUserMedia({ video: true });

        // Enumerate devices with proper labels
        const devices = await navigator.mediaDevices.enumerateDevices();
        cameraSystem.devices = devices.filter(d => d.kind === 'videoinput');

        if (cameraSystem.devices.length < 2) {
            throw new Error(`Only ${cameraSystem.devices.length} cameras detected (need 3)`);
        }

        // Initialize each camera with UNIQUE settings
        await initCamera(0, {
            width: 1280, height: 720,
            facingMode: 'environment'
        });
        await new Promise(resolve => setTimeout(resolve, 1000));

        await initCamera(1, {
            width: 640, height: 480,
            facingMode: 'user'
        });
        await new Promise(resolve => setTimeout(resolve, 1000));

        // Verify all streams are active
        const activeStreams = cameraSystem.streams.filter(s => s.active);
        if (activeStreams.length === 2) {
            status.textContent = "✅ All cameras working";
            status.style.color = "#4CAF50";
        } else {
            throw new Error(`${activeStreams.length}/3 cameras initialized`);
        }

    } catch (err) {
        status.textContent = "❌ Error: " + err.message;
        status.style.color = "red";
    }
}

async function initCamera(index, customConstraints = {}) {
    const video = document.getElementById(`video${index+1}`);
    const camElement = document.getElementById(`cam${index+1}`);
    const label = camElement.querySelector('.camera-label');

    try {
        const device = cameraSystem.devices[index];

        // Force different parameters for identical cameras
        const constraints = {
            video: {
                deviceId: { exact: device.deviceId },
                ...customConstraints,
                frameRate: { ideal: [30, 25, 20][index] } // Different FPS
            }
        };

        const stream = await navigator.mediaDevices.getUserMedia(constraints);

        // Verify this is a new stream (not duplicate)
        if (cameraSystem.streams.some(s =>
            s.getVideoTracks()[0].getSettings().deviceId ===
            stream.getVideoTracks()[0].getSettings().deviceId
        )) {
            throw new Error("Duplicate camera stream detected");
        }

        video.srcObject = stream;
        cameraSystem.streams.push(stream);

        camElement.style.border = "2px solid #4CAF50";
        label.textContent = `Cam ${index+1}: ${device.label || 'Device '+index}`;

    } catch (err) {
        console.error(`Camera ${index+1} error:`, err);
        camElement.innerHTML = `
            <div class="camera-error">
                ${err.message.includes("Duplicate") ?
                 'Identical Camera Conflict' :
                 `Camera ${index+1} Error`}
            </div>
        `;
        camElement.style.border = "2px solid #F44336";
        throw err; // Propagate error
    }
}

// Start initialization
setTimeout(setupCameras, 500);

// Retry button
document.getElementById('retry-btn').addEventListener('click', setupCameras);
</script>
"""))

In [105]:
from IPython.display import display, HTML

display(HTML("""
<div style="margin: 20px; text-align: center;">
    <h2>Multi Objects Tracking using Multi Cameras</h2>
    <div id="camera-container" style="display: flex; justify-content: center; gap: 20px; flex-wrap: wrap;">
        <!-- Camera 1 -->
        <div id="cam1" style="width: 320px; height: 240px; border: 2px solid #ddd; background: #000; overflow: hidden; position: relative;">
            <video id="video1" autoplay playsinline muted style="width: 100%; height: 100%; object-fit: cover;"></video>
            <canvas id="canvas1" style="position: absolute; top: 0; left: 0; width: 100%; height: 100%;"></canvas>
            <div class="camera-label">Camera 1</div>
            <div class="object-count" style="position: absolute; bottom: 10px; left: 10px; background: rgba(0,0,0,0.7); color: white; padding: 2px 8px; border-radius: 4px;">
                Persons: 0 | Cars: 0
            </div>
        </div>
        <!-- Camera 2 -->
        <div id="cam2" style="width: 320px; height: 240px; border: 2px solid #ddd; background: #000; overflow: hidden; position: relative;">
            <video id="video2" autoplay playsinline muted style="width: 100%; height: 100%; object-fit: cover;"></video>
            <canvas id="canvas2" style="position: absolute; top: 0; left: 0; width: 100%; height: 100%;"></canvas>
            <div class="camera-label">Camera 2</div>
            <div class="object-count" style="position: absolute; bottom: 10px; left: 10px; background: rgba(0,0,0,0.7); color: white; padding: 2px 8px; border-radius: 4px;">
                Persons: 0 | Cars: 0
            </div>
        </div>
    </div>
    <div id="tracking-info" style="margin-top: 20px; padding: 15px; background: #f5f5f5; border-radius: 5px; text-align: left;">
        <h3 style="margin-top: 0;">Tracking Information</h3>
        <div id="tracking-log" style="max-height: 150px; overflow-y: auto;"></div>
    </div>
    <p id="status" style="color: #666; margin-top: 20px;">Initializing cameras...</p>
    <button id="retry-btn" style="padding: 10px 20px; background: #2196F3; color: white; border: none; border-radius: 5px; margin-top: 15px;">Reinitialize Cameras</button>
    <button id="start-tracking-btn" style="padding: 10px 20px; background: #4CAF50; color: white; border: none; border-radius: 5px; margin-top: 15px; margin-left: 10px;">Start Tracking</button>
</div>

<style>
.camera-label {
    position: absolute;
    top: 10px;
    left: 10px;
    background: rgba(0,0,0,0.7);
    color: white;
    padding: 2px 8px;
    border-radius: 4px;
}
.camera-error {
    width:100%;
    height:100%;
    display:flex;
    align-items:center;
    justify-content:center;
    background:#222;
    color:white;
}
.tracking-box {
    position: absolute;
    border: 2px solid;
    font-size: 10px;
    font-weight: bold;
    padding: 1px;
    background: rgba(0,0,0,0.3);
}
.person-box {
    border-color: #FF5722;
    color: #FF5722;
}
.car-box {
    border-color: #3F51B5;
    color: #3F51B5;
}
</style>

<script>
// Load TensorFlow.js and COCO-SSD model
async function loadModel() {
    const status = document.getElementById('status');
    status.textContent = "⌛ Loading object detection model...";

    // Load TensorFlow.js and COCO-SSD model
    const script = document.createElement('script');
    script.src = "https://cdn.jsdelivr.net/npm/@tensorflow/tfjs";
    document.head.appendChild(script);

    const script2 = document.createElement('script');
    script2.src = "https://cdn.jsdelivr.net/npm/@tensorflow-models/coco-ssd";
    document.head.appendChild(script2);

    await new Promise(resolve => {
        script.onload = resolve;
    });

    await new Promise(resolve => {
        script2.onload = resolve;
    });

    //@ts-ignore
    const model = await cocoSsd.load();
    status.textContent = "✅ Object detection model loaded";
    return model;
}

const cameraSystem = {
    streams: [],
    devices: [],
    model: null,
    trackingActive: false,
    trackedObjects: [[], []], // Tracked objects for each camera
    objectColors: {} // Colors for consistent tracking
};

// Generate a random color for each tracked object
function getObjectColor(trackId) {
    if (!cameraSystem.objectColors[trackId]) {
        const hue = Math.floor(Math.random() * 360);
        cameraSystem.objectColors[trackId] = `hsl(${hue}, 100%, 50%)`;
    }
    return cameraSystem.objectColors[trackId];
}

// Calculate Intersection over Union (IoU)
function calculateIOU(box1, box2) {
    const x1 = Math.max(box1.x, box2.x);
    const y1 = Math.max(box1.y, box2.y);
    const x2 = Math.min(box1.x + box1.width, box2.x + box2.width);
    const y2 = Math.min(box1.y + box1.height, box2.y + box2.height);

    const intersection = Math.max(0, x2 - x1) * Math.max(0, y2 - y1);
    const area1 = box1.width * box1.height;
    const area2 = box2.width * box2.height;

    return intersection / (area1 + area2 - intersection);
}

// Track objects between frames
function updateTrackedObjects(cameraIndex, detections) {
    const currentTime = Date.now();
    const tracked = cameraSystem.trackedObjects[cameraIndex];

    // Mark all current tracked objects as unmatched
    tracked.forEach(obj => {
        obj.matched = false;
        obj.age = (currentTime - obj.lastSeen) / 1000; // Age in seconds
    });

    // Process new detections
    detections.forEach(det => {
        let bestMatch = null;
        let bestIOU = 0.3; // Minimum IOU threshold

        // Find best matching existing object
        tracked.forEach(obj => {
            if (obj.class !== det.class) return; // Only match same class

            const iou = calculateIOU(obj, det);
            if (iou > bestIOU) {
                bestIOU = iou;
                bestMatch = obj;
            }
        });

        if (bestMatch) {
            // Update existing object
            bestMatch.x = det.x;
            bestMatch.y = det.y;
            bestMatch.width = det.width;
            bestMatch.height = det.height;
            bestMatch.score = det.score;
            bestMatch.lastSeen = currentTime;
            bestMatch.matched = true;
            bestMatch.age = 0;
        } else {
            // Create new tracked object
            const newObj = {
                ...det,
                id: `${cameraIndex}-${det.class}-${currentTime}-${Math.random().toString(36).substr(2, 5)}`,
                lastSeen: currentTime,
                matched: true,
                age: 0
            };
            tracked.push(newObj);
        }
    });

    // Remove objects that haven't been matched for a while
    cameraSystem.trackedObjects[cameraIndex] = tracked.filter(obj => {
        return obj.matched || obj.age < 2; // Keep if matched or less than 2 seconds old
    });
}

// Draw bounding boxes and tracking info
function drawDetections(cameraIndex) {
    const canvas = document.getElementById(`canvas${cameraIndex+1}`);
    const video = document.getElementById(`video${cameraIndex+1}`);
    const ctx = canvas.getContext('2d');

    // Set canvas dimensions to match video
    canvas.width = video.videoWidth;
    canvas.height = video.videoHeight;

    // Clear canvas
    ctx.clearRect(0, 0, canvas.width, canvas.height);

    // Count objects by class
    const counts = { person: 0, car: 0 };

    // Draw each tracked object
    cameraSystem.trackedObjects[cameraIndex].forEach(obj => {
        if (obj.class === 'person' || obj.class === 'car') {
            counts[obj.class]++;

            const color = getObjectColor(obj.id);
            const className = obj.class === 'person' ? 'person-box' : 'car-box';

            // Draw bounding box
            ctx.strokeStyle = color;
            ctx.lineWidth = 2;
            ctx.strokeRect(obj.x, obj.y, obj.width, obj.height);

            // Draw label background
            ctx.fillStyle = 'rgba(0, 0, 0, 0.5)';
            ctx.fillRect(obj.x, obj.y - 20, ctx.measureText(`${obj.class} ${(obj.score * 100).toFixed(1)}%`).width + 10, 20);

            // Draw label text
            ctx.fillStyle = color;
            ctx.font = '12px Arial';
            ctx.fillText(`${obj.class} ${(obj.score * 100).toFixed(1)}%`, obj.x + 5, obj.y - 5);

            // Draw ID
            ctx.fillStyle = 'white';
            ctx.font = '10px Arial';
            ctx.fillText(`ID: ${obj.id.split('-').pop()}`, obj.x + 5, obj.y + obj.height - 5);
        }
    });

    // Update object count display
    const countElement = document.querySelector(`#cam${cameraIndex+1} .object-count`);
    countElement.textContent = `Persons: ${counts.person} | Cars: ${counts.car}`;
}

// Process video frames for object detection
async function detectObjects(cameraIndex) {
    if (!cameraSystem.trackingActive) return;

    const video = document.getElementById(`video${cameraIndex+1}`);
    const canvas = document.getElementById(`canvas${cameraIndex+1}`);

    if (video.readyState < 2 || !cameraSystem.model) return;

    try {
        //@ts-ignore
        const predictions = await cameraSystem.model.detect(video);

        // Filter for persons and cars only
        const filtered = predictions.filter(p =>
            p.class === 'person' || p.class === 'car'
        ).map(p => ({
            x: p.bbox[0],
            y: p.bbox[1],
            width: p.bbox[2],
            height: p.bbox[3],
            class: p.class,
            score: p.score
        }));

        // Update tracked objects
        updateTrackedObjects(cameraIndex, filtered);

        // Draw detections
        drawDetections(cameraIndex);

        // Log new objects
        filtered.forEach(det => {
            const isNew = !cameraSystem.trackedObjects[cameraIndex].some(
                obj => obj.id && obj.id.startsWith(`${cameraIndex}-${det.class}`) &&
                       calculateIOU(obj, det) > 0.3
            );

            if (isNew) {
                const logElement = document.getElementById('tracking-log');
                const entry = document.createElement('div');
                entry.textContent = `[Cam ${cameraIndex+1}] New ${det.class} detected (${(det.score * 100).toFixed(1)}% confidence)`;
                logElement.appendChild(entry);
                logElement.scrollTop = logElement.scrollHeight;
            }
        });

    } catch (err) {
        console.error(`Detection error for camera ${cameraIndex}:`, err);
    }

    // Continue processing
    requestAnimationFrame(() => detectObjects(cameraIndex));
}

async function setupCameras() {
    const status = document.getElementById('status');
    status.textContent = "⌛ Detecting cameras...";
    status.style.color = "#666";

    // Clear previous streams
    cameraSystem.streams.forEach(stream => {
        stream.getTracks().forEach(track => track.stop());
    });
    cameraSystem.streams = [];
    cameraSystem.trackedObjects = [[], []];

    try {
        // First get basic access
        await navigator.mediaDevices.getUserMedia({ video: true });

        // Enumerate devices with proper labels
        const devices = await navigator.mediaDevices.enumerateDevices();
        cameraSystem.devices = devices.filter(d => d.kind === 'videoinput');

        if (cameraSystem.devices.length < 2) {
            throw new Error(`Only ${cameraSystem.devices.length} cameras detected (need at least 2)`);
        }

        // Initialize each camera
        await initCamera(0, {
            width: { ideal: 1280 },
            height: { ideal: 720 },
            facingMode: 'environment'
        });
        await new Promise(resolve => setTimeout(resolve, 500));

        await initCamera(1, {
            width: { ideal: 640 },
            height: { ideal: 480 },
            facingMode: 'user'
        });

        // Verify all streams are active
        const activeStreams = cameraSystem.streams.filter(s => s.active);
        if (activeStreams.length === 2) {
            status.textContent = "✅ All cameras working";
            status.style.color = "#4CAF50";

            // Load model if not already loaded
            if (!cameraSystem.model) {
                //@ts-ignore
                if (typeof cocoSsd !== 'undefined') {
                    //@ts-ignore
                    cameraSystem.model = await cocoSsd.load();
                    status.textContent = "✅ Cameras ready & model loaded";
                }
            }
        } else {
            throw new Error(`${activeStreams.length}/2 cameras initialized`);
        }

    } catch (err) {
        status.textContent = "❌ Error: " + err.message;
        status.style.color = "red";
    }
}

async function initCamera(index, customConstraints = {}) {
    const video = document.getElementById(`video${index+1}`);
    const camElement = document.getElementById(`cam${index+1}`);
    const label = camElement.querySelector('.camera-label');

    try {
        const device = cameraSystem.devices[index];

        const constraints = {
            video: {
                deviceId: device.deviceId ? { exact: device.deviceId } : undefined,
                ...customConstraints
            }
        };

        const stream = await navigator.mediaDevices.getUserMedia(constraints);

        // Verify this is a new stream
        if (cameraSystem.streams.some(s =>
            s.getVideoTracks()[0].getSettings().deviceId ===
            stream.getVideoTracks()[0].getSettings().deviceId
        )) {
            throw new Error("Duplicate camera stream detected");
        }

        video.srcObject = stream;
        cameraSystem.streams.push(stream);

        camElement.style.border = "2px solid #4CAF50";
        label.textContent = `Cam ${index+1}: ${device.label || 'Device '+index}`;

    } catch (err) {
        console.error(`Camera ${index+1} error:`, err);
        camElement.innerHTML = `
            <div class="camera-error">
                ${err.message.includes("Duplicate") ?
                 'Identical Camera Conflict' :
                 `Camera ${index+1} Error`}
            </div>
        `;
        camElement.style.border = "2px solid #F44336";
        throw err;
    }
}

// Start initialization
setTimeout(async () => {
    await setupCameras();

    // Load model if not already loaded by setupCameras
    //@ts-ignore
    if (typeof cocoSsd !== 'undefined' && !cameraSystem.model) {
        //@ts-ignore
        cameraSystem.model = await cocoSsd.load();
        document.getElementById('status').textContent = "✅ Cameras ready & model loaded";
    }
}, 500);

// Retry button
document.getElementById('retry-btn').addEventListener('click', setupCameras);

// Start tracking button
document.getElementById('start-tracking-btn').addEventListener('click', () => {
    if (!cameraSystem.model) {
        alert("Please wait for the model to load first");
        return;
    }

    cameraSystem.trackingActive = true;
    document.getElementById('status').textContent = "🔍 Tracking persons and cars...";

    // Start detection for each camera
    for (let i = 0; i < cameraSystem.streams.length; i++) {
        detectObjects(i);
    }
});
</script>
"""))

In [106]:
from IPython.display import display, HTML

display(HTML("""
<div style="margin: 20px; text-align: center;">
    <h2>Multi Objects Tracking using Multi Cameras</h2>
    <div id="camera-container" style="display: flex; justify-content: center; gap: 20px; flex-wrap: wrap;">
        <!-- Camera 1 -->
        <div id="cam1" style="width: 320px; height: 240px; border: 2px solid #ddd; background: #000; overflow: hidden; position: relative;">
            <video id="video1" autoplay playsinline muted style="width: 100%; height: 100%; object-fit: cover;"></video>
            <canvas id="canvas1" style="position: absolute; top: 0; left: 0; width: 100%; height: 100%;"></canvas>
            <div class="camera-label">Camera 1</div>
            <div class="object-count" style="position: absolute; bottom: 10px; left: 10px; background: rgba(0,0,0,0.7); color: white; padding: 2px 8px; border-radius: 4px;">
                Persons: 0 | Cars: 0
            </div>
        </div>
        <!-- Camera 2 -->
        <div id="cam2" style="width: 320px; height: 240px; border: 2px solid #ddd; background: #000; overflow: hidden; position: relative;">
            <video id="video2" autoplay playsinline muted style="width: 100%; height: 100%; object-fit: cover;"></video>
            <canvas id="canvas2" style="position: absolute; top: 0; left: 0; width: 100%; height: 100%;"></canvas>
            <div class="camera-label">Camera 2</div>
            <div class="object-count" style="position: absolute; bottom: 10px; left: 10px; background: rgba(0,0,0,0.7); color: white; padding: 2px 8px; border-radius: 4px;">
                Persons: 0 | Cars: 0
            </div>
        </div>
    </div>
    <div id="tracking-info" style="margin-top: 20px; padding: 15px; background: #f5f5f5; border-radius: 5px; text-align: left;">
        <h3 style="margin-top: 0;">Tracking Information</h3>
        <div id="tracking-log" style="max-height: 150px; overflow-y: auto;"></div>
    </div>
    <p id="status" style="color: #666; margin-top: 20px;">Initializing cameras...</p>
    <div>
        <button id="retry-btn" style="padding: 10px 20px; background: #2196F3; color: white; border: none; border-radius: 5px; margin-top: 15px;">Reinitialize Cameras</button>
        <button id="start-tracking-btn" style="padding: 10px 20px; background: #4CAF50; color: white; border: none; border-radius: 5px; margin-top: 15px; margin-left: 10px;">Start Tracking</button>
        <button id="stop-tracking-btn" style="padding: 10px 20px; background: #F44336; color: white; border: none; border-radius: 5px; margin-top: 15px; margin-left: 10px; display: none;">Stop Tracking</button>
    </div>
</div>

<style>
.camera-label {
    position: absolute;
    top: 10px;
    left: 10px;
    background: rgba(0,0,0,0.7);
    color: white;
    padding: 2px 8px;
    border-radius: 4px;
}
.camera-error {
    width:100%;
    height:100%;
    display:flex;
    align-items:center;
    justify-content:center;
    background:#222;
    color:white;
}
.tracking-box {
    position: absolute;
    border: 2px solid;
    font-size: 10px;
    font-weight: bold;
    padding: 1px;
    background: rgba(0,0,0,0.3);
}
.person-box {
    border-color: #FF5722;
    color: #FF5722;
}
.car-box {
    border-color: #3F51B5;
    color: #3F51B5;
}
.loading-spinner {
    border: 4px solid #f3f3f3;
    border-top: 4px solid #3498db;
    border-radius: 50%;
    width: 30px;
    height: 30px;
    animation: spin 2s linear infinite;
    margin: 20px auto;
}
@keyframes spin {
    0% { transform: rotate(0deg); }
    100% { transform: rotate(360deg); }
}
</style>

<script>
const cameraSystem = {
    streams: [],
    devices: [],
    model: null,
    trackingActive: false,
    modelLoading: false,
    trackedObjects: [[], []],
    objectColors: {}
};

// Generate a random color for each tracked object
function getObjectColor(trackId) {
    if (!cameraSystem.objectColors[trackId]) {
        const hue = Math.floor(Math.random() * 360);
        cameraSystem.objectColors[trackId] = `hsl(${hue}, 100%, 50%)`;
    }
    return cameraSystem.objectColors[trackId];
}

// Calculate Intersection over Union (IoU)
function calculateIOU(box1, box2) {
    const x1 = Math.max(box1.x, box2.x);
    const y1 = Math.max(box1.y, box2.y);
    const x2 = Math.min(box1.x + box1.width, box2.x + box2.width);
    const y2 = Math.min(box1.y + box1.height, box2.y + box2.height);

    const intersection = Math.max(0, x2 - x1) * Math.max(0, y2 - y1);
    const area1 = box1.width * box1.height;
    const area2 = box2.width * box2.height;

    return intersection / (area1 + area2 - intersection);
}

// Track objects between frames
function updateTrackedObjects(cameraIndex, detections) {
    const currentTime = Date.now();
    const tracked = cameraSystem.trackedObjects[cameraIndex];

    // Mark all current tracked objects as unmatched
    tracked.forEach(obj => {
        obj.matched = false;
        obj.age = (currentTime - obj.lastSeen) / 1000; // Age in seconds
    });

    // Process new detections
    detections.forEach(det => {
        let bestMatch = null;
        let bestIOU = 0.3; // Minimum IOU threshold

        // Find best matching existing object
        tracked.forEach(obj => {
            if (obj.class !== det.class) return; // Only match same class

            const iou = calculateIOU(obj, det);
            if (iou > bestIOU) {
                bestIOU = iou;
                bestMatch = obj;
            }
        });

        if (bestMatch) {
            // Update existing object
            bestMatch.x = det.x;
            bestMatch.y = det.y;
            bestMatch.width = det.width;
            bestMatch.height = det.height;
            bestMatch.score = det.score;
            bestMatch.lastSeen = currentTime;
            bestMatch.matched = true;
            bestMatch.age = 0;
        } else {
            // Create new tracked object
            const newObj = {
                ...det,
                id: `${cameraIndex}-${det.class}-${currentTime}-${Math.random().toString(36).substr(2, 5)}`,
                lastSeen: currentTime,
                matched: true,
                age: 0
            };
            tracked.push(newObj);
        }
    });

    // Remove objects that haven't been matched for a while
    cameraSystem.trackedObjects[cameraIndex] = tracked.filter(obj => {
        return obj.matched || obj.age < 2; // Keep if matched or less than 2 seconds old
    });
}

// Draw bounding boxes and tracking info
function drawDetections(cameraIndex) {
    const canvas = document.getElementById(`canvas${cameraIndex+1}`);
    const video = document.getElementById(`video${cameraIndex+1}`);
    const ctx = canvas.getContext('2d');

    // Set canvas dimensions to match video
    canvas.width = video.videoWidth;
    canvas.height = video.videoHeight;

    // Clear canvas
    ctx.clearRect(0, 0, canvas.width, canvas.height);

    // Count objects by class
    const counts = { person: 0, car: 0 };

    // Draw each tracked object
    cameraSystem.trackedObjects[cameraIndex].forEach(obj => {
        if (obj.class === 'person' || obj.class === 'car') {
            counts[obj.class]++;

            const color = getObjectColor(obj.id);
            const className = obj.class === 'person' ? 'person-box' : 'car-box';

            // Draw bounding box
            ctx.strokeStyle = color;
            ctx.lineWidth = 2;
            ctx.strokeRect(obj.x, obj.y, obj.width, obj.height);

            // Draw label background
            ctx.fillStyle = 'rgba(0, 0, 0, 0.5)';
            ctx.fillRect(obj.x, obj.y - 20, ctx.measureText(`${obj.class} ${(obj.score * 100).toFixed(1)}%`).width + 10, 20);

            // Draw label text
            ctx.fillStyle = color;
            ctx.font = '12px Arial';
            ctx.fillText(`${obj.class} ${(obj.score * 100).toFixed(1)}%`, obj.x + 5, obj.y - 5);

            // Draw ID
            ctx.fillStyle = 'white';
            ctx.font = '10px Arial';
            ctx.fillText(`ID: ${obj.id.split('-').pop()}`, obj.x + 5, obj.y + obj.height - 5);
        }
    });

    // Update object count display
    const countElement = document.querySelector(`#cam${cameraIndex+1} .object-count`);
    countElement.textContent = `Persons: ${counts.person} | Cars: ${counts.car}`;
}

// Process video frames for object detection
async function detectObjects(cameraIndex) {
    if (!cameraSystem.trackingActive) return;

    const video = document.getElementById(`video${cameraIndex+1}`);
    const canvas = document.getElementById(`canvas${cameraIndex+1}`);

    if (video.readyState < 2 || !cameraSystem.model) return;

    try {
        //@ts-ignore
        const predictions = await cameraSystem.model.detect(video);

        // Filter for persons and cars only
        const filtered = predictions.filter(p =>
            p.class === 'person' || p.class === 'car'
        ).map(p => ({
            x: p.bbox[0],
            y: p.bbox[1],
            width: p.bbox[2],
            height: p.bbox[3],
            class: p.class,
            score: p.score
        }));

        // Update tracked objects
        updateTrackedObjects(cameraIndex, filtered);

        // Draw detections
        drawDetections(cameraIndex);

        // Log new objects
        filtered.forEach(det => {
            const isNew = !cameraSystem.trackedObjects[cameraIndex].some(
                obj => obj.id && obj.id.startsWith(`${cameraIndex}-${det.class}`) &&
                       calculateIOU(obj, det) > 0.3
            );

            if (isNew) {
                const logElement = document.getElementById('tracking-log');
                const entry = document.createElement('div');
                entry.textContent = `[Cam ${cameraIndex+1}] New ${det.class} detected (${(det.score * 100).toFixed(1)}% confidence)`;
                logElement.appendChild(entry);
                logElement.scrollTop = logElement.scrollHeight;
            }
        });

    } catch (err) {
        console.error(`Detection error for camera ${cameraIndex}:`, err);
    }

    // Continue processing
    requestAnimationFrame(() => detectObjects(cameraIndex));
}

async function loadModel() {
    const status = document.getElementById('status');

    if (cameraSystem.model) return true; // Model already loaded

    try {
        cameraSystem.modelLoading = true;
        status.innerHTML = '<div class="loading-spinner"></div> Loading object detection model...';

        // Check if TensorFlow.js and COCO-SSD are available
        //@ts-ignore
        if (typeof tf === 'undefined' || typeof cocoSsd === 'undefined') {
            // Load TensorFlow.js
            await new Promise((resolve) => {
                const script = document.createElement('script');
                script.src = "https://cdn.jsdelivr.net/npm/@tensorflow/tfjs";
                document.head.appendChild(script);
                script.onload = resolve;
            });

            // Load COCO-SSD
            await new Promise((resolve) => {
                const script = document.createElement('script');
                script.src = "https://cdn.jsdelivr.net/npm/@tensorflow-models/coco-ssd";
                document.head.appendChild(script);
                script.onload = resolve;
            });
        }

        //@ts-ignore
        cameraSystem.model = await cocoSsd.load();
        cameraSystem.modelLoading = false;
        status.textContent = "✅ Object detection model loaded and ready";
        return true;
    } catch (err) {
        console.error("Model loading error:", err);
        status.textContent = "❌ Failed to load object detection model";
        cameraSystem.modelLoading = false;
        return false;
    }
}

async function setupCameras() {
    const status = document.getElementById('status');
    status.textContent = "⌛ Detecting cameras...";
    status.style.color = "#666";

    // Clear previous streams
    cameraSystem.streams.forEach(stream => {
        stream.getTracks().forEach(track => track.stop());
    });
    cameraSystem.streams = [];
    cameraSystem.trackedObjects = [[], []];

    try {
        // First get basic access
        await navigator.mediaDevices.getUserMedia({ video: true });

        // Enumerate devices with proper labels
        const devices = await navigator.mediaDevices.enumerateDevices();
        cameraSystem.devices = devices.filter(d => d.kind === 'videoinput');

        if (cameraSystem.devices.length < 2) {
            throw new Error(`Only ${cameraSystem.devices.length} cameras detected (need at least 2)`);
        }

        // Initialize each camera
        await initCamera(0, {
            width: { ideal: 1280 },
            height: { ideal: 720 },
            facingMode: 'environment'
        });
        await new Promise(resolve => setTimeout(resolve, 500));

        await initCamera(1, {
            width: { ideal: 640 },
            height: { ideal: 480 },
            facingMode: 'user'
        });

        // Verify all streams are active
        const activeStreams = cameraSystem.streams.filter(s => s.active);
        if (activeStreams.length === 2) {
            status.textContent = "✅ All cameras working";
            status.style.color = "#4CAF50";
        } else {
            throw new Error(`${activeStreams.length}/2 cameras initialized`);
        }

    } catch (err) {
        status.textContent = "❌ Error: " + err.message;
        status.style.color = "red";
    }
}

async function initCamera(index, customConstraints = {}) {
    const video = document.getElementById(`video${index+1}`);
    const camElement = document.getElementById(`cam${index+1}`);
    const label = camElement.querySelector('.camera-label');

    try {
        const device = cameraSystem.devices[index];

        const constraints = {
            video: {
                deviceId: device.deviceId ? { exact: device.deviceId } : undefined,
                ...customConstraints
            }
        };

        const stream = await navigator.mediaDevices.getUserMedia(constraints);

        // Verify this is a new stream
        if (cameraSystem.streams.some(s =>
            s.getVideoTracks()[0].getSettings().deviceId ===
            stream.getVideoTracks()[0].getSettings().deviceId
        )) {
            throw new Error("Duplicate camera stream detected");
        }

        video.srcObject = stream;
        cameraSystem.streams.push(stream);

        camElement.style.border = "2px solid #4CAF50";
        label.textContent = `Cam ${index+1}: ${device.label || 'Device '+index}`;

    } catch (err) {
        console.error(`Camera ${index+1} error:`, err);
        camElement.innerHTML = `
            <div class="camera-error">
                ${err.message.includes("Duplicate") ?
                 'Identical Camera Conflict' :
                 `Camera ${index+1} Error`}
            </div>
        `;
        camElement.style.border = "2px solid #F44336";
        throw err;
    }
}

// Start tracking function
async function startTracking() {
    const status = document.getElementById('status');
    const startBtn = document.getElementById('start-tracking-btn');
    const stopBtn = document.getElementById('stop-tracking-btn');

    // Disable start button during loading
    startBtn.disabled = true;
    startBtn.textContent = "Loading...";
    startBtn.style.background = "#9E9E9E";

    // Show loading status
    status.innerHTML = '<div class="loading-spinner"></div> Loading object detection model...';

    try {
        // Load model if not already loaded
        const modelLoaded = await loadModel();

        if (!modelLoaded) {
            status.textContent = "❌ Failed to load object detection model";
            startBtn.disabled = false;
            startBtn.textContent = "Start Tracking";
            startBtn.style.background = "#4CAF50";
            return;
        }

        // Start tracking
        cameraSystem.trackingActive = true;
        status.textContent = "🔍 Tracking persons and cars...";

        // Start detection for each camera
        for (let i = 0; i < cameraSystem.streams.length; i++) {
            detectObjects(i);
        }

        // Toggle buttons
        startBtn.style.display = "none";
        stopBtn.style.display = "inline-block";

    } catch (err) {
        console.error("Tracking start error:", err);
        status.textContent = "❌ Error starting tracking: " + err.message;
        startBtn.disabled = false;
        startBtn.textContent = "Start Tracking";
        startBtn.style.background = "#4CAF50";
    }
}

// Stop tracking function
function stopTracking() {
    cameraSystem.trackingActive = false;
    document.getElementById('status').textContent = "✅ Tracking stopped";

    // Toggle buttons
    document.getElementById('start-tracking-btn').style.display = "inline-block";
    document.getElementById('stop-tracking-btn').style.display = "none";

    // Reset start button
    const startBtn = document.getElementById('start-tracking-btn');
    startBtn.disabled = false;
    startBtn.textContent = "Start Tracking";
    startBtn.style.background = "#4CAF50";
}

// Start initialization
setTimeout(async () => {
    await setupCameras();
}, 500);

// Event listeners
document.getElementById('retry-btn').addEventListener('click', setupCameras);
document.getElementById('start-tracking-btn').addEventListener('click', startTracking);
document.getElementById('stop-tracking-btn').addEventListener('click', stopTracking);
</script>
"""))

In [107]:
from IPython.display import display, HTML

display(HTML("""
<div style="margin: 20px; text-align: center;">
    <h2>Multi Objects Tracking using Multi Cameras</h2>
    <div id="camera-container" style="display: flex; justify-content: center; gap: 20px; flex-wrap: wrap;">
        <!-- Camera 1 -->
        <div id="cam1" style="width: 320px; height: 240px; border: 2px solid #ddd; background: #000; overflow: hidden; position: relative;">
            <video id="video1" autoplay playsinline muted style="width: 100%; height: 100%; object-fit: cover;"></video>
            <canvas id="canvas1" style="position: absolute; top: 0; left: 0; width: 100%; height: 100%;"></canvas>
            <div class="camera-label">Camera 1</div>
            <div class="object-count" style="position: absolute; bottom: 10px; left: 10px; background: rgba(0,0,0,0.7); color: white; padding: 2px 8px; border-radius: 4px;">
                Persons: 0 | Cars: 0
            </div>
        </div>
        <!-- Camera 2 -->
        <div id="cam2" style="width: 320px; height: 240px; border: 2px solid #ddd; background: #000; overflow: hidden; position: relative;">
            <video id="video2" autoplay playsinline muted style="width: 100%; height: 100%; object-fit: cover;"></video>
            <canvas id="canvas2" style="position: absolute; top: 0; left: 0; width: 100%; height: 100%;"></canvas>
            <div class="camera-label">Camera 2</div>
            <div class="object-count" style="position: absolute; bottom: 10px; left: 10px; background: rgba(0,0,0,0.7); color: white; padding: 2px 8px; border-radius: 4px;">
                Persons: 0 | Cars: 0
            </div>
        </div>
    </div>
    <div id="tracking-info" style="margin-top: 20px; padding: 15px; background: #f5f5f5; border-radius: 5px; text-align: left;">
        <div style="display: flex; justify-content: space-between; align-items: center;">
            <h3 style="margin-top: 0;">Tracking Information</h3>
            <button id="clear-log-btn" style="padding: 5px 10px; background: #f44336; color: white; border: none; border-radius: 3px;">Clear Log</button>
        </div>
        <div id="tracking-log" style="max-height: 200px; overflow-y: auto; font-family: monospace; background: white; padding: 10px; border: 1px solid #ddd; border-radius: 4px;"></div>
        <div style="margin-top: 10px; display: flex; gap: 10px;">
            <div style="flex: 1; background: #FF5722; color: white; padding: 5px; border-radius: 3px;">Person</div>
            <div style="flex: 1; background: #3F51B5; color: white; padding: 5px; border-radius: 3px;">Car</div>
            <div style="flex: 2; background: #4CAF50; color: white; padding: 5px; border-radius: 3px;">Active Objects: <span id="total-objects">0</span></div>
        </div>
    </div>
    <p id="status" style="color: #666; margin-top: 20px;">Initializing cameras...</p>
    <div>
        <button id="retry-btn" style="padding: 10px 20px; background: #2196F3; color: white; border: none; border-radius: 5px; margin-top: 15px;">Reinitialize Cameras</button>
        <button id="start-tracking-btn" style="padding: 10px 20px; background: #4CAF50; color: white; border: none; border-radius: 5px; margin-top: 15px; margin-left: 10px;">Start Tracking</button>
        <button id="stop-tracking-btn" style="padding: 10px 20px; background: #F44336; color: white; border: none; border-radius: 5px; margin-top: 15px; margin-left: 10px; display: none;">Stop Tracking</button>
    </div>
</div>

<style>
.camera-label {
    position: absolute;
    top: 10px;
    left: 10px;
    background: rgba(0,0,0,0.7);
    color: white;
    padding: 2px 8px;
    border-radius: 4px;
}
.camera-error {
    width:100%;
    height:100%;
    display:flex;
    align-items:center;
    justify-content:center;
    background:#222;
    color:white;
}
.tracking-box {
    position: absolute;
    border: 2px solid;
    font-size: 10px;
    font-weight: bold;
    padding: 1px;
    background: rgba(0,0,0,0.3);
}
.person-box {
    border-color: #FF5722;
    color: #FF5722;
}
.car-box {
    border-color: #3F51B5;
    color: #3F51B5;
}
.loading-spinner {
    border: 4px solid #f3f3f3;
    border-top: 4px solid #3498db;
    border-radius: 50%;
    width: 30px;
    height: 30px;
    animation: spin 2s linear infinite;
    margin: 20px auto;
}
@keyframes spin {
    0% { transform: rotate(0deg); }
    100% { transform: rotate(360deg); }
}
.log-entry {
    margin: 3px 0;
    padding: 3px;
    border-bottom: 1px solid #eee;
}
.log-timestamp {
    color: #666;
    margin-right: 8px;
}
.log-camera {
    font-weight: bold;
    margin-right: 8px;
}
.log-person {
    color: #FF5722;
}
.log-car {
    color: #3F51B5;
}
</style>

<script>
const cameraSystem = {
    streams: [],
    devices: [],
    model: null,
    trackingActive: false,
    modelLoading: false,
    trackedObjects: [[], []],
    objectColors: {},
    detectionHistory: []
};

// Format timestamp for logging
function formatTimestamp() {
    const now = new Date();
    return now.toLocaleTimeString() + '.' + now.getMilliseconds().toString().padStart(3, '0');
}

// Generate a random color for each tracked object
function getObjectColor(trackId) {
    if (!cameraSystem.objectColors[trackId]) {
        const hue = Math.floor(Math.random() * 360);
        cameraSystem.objectColors[trackId] = `hsl(${hue}, 100%, 50%)`;
    }
    return cameraSystem.objectColors[trackId];
}

// Calculate Intersection over Union (IoU)
function calculateIOU(box1, box2) {
    const x1 = Math.max(box1.x, box2.x);
    const y1 = Math.max(box1.y, box2.y);
    const x2 = Math.min(box1.x + box1.width, box2.x + box2.width);
    const y2 = Math.min(box1.y + box1.height, box2.y + box2.height);

    const intersection = Math.max(0, x2 - x1) * Math.max(0, y2 - y1);
    const area1 = box1.width * box1.height;
    const area2 = box2.width * box2.height;

    return intersection / (area1 + area2 - intersection);
}

// Track objects between frames
function updateTrackedObjects(cameraIndex, detections) {
    const currentTime = Date.now();
    const tracked = cameraSystem.trackedObjects[cameraIndex];

    // Mark all current tracked objects as unmatched
    tracked.forEach(obj => {
        obj.matched = false;
        obj.age = (currentTime - obj.lastSeen) / 1000; // Age in seconds
    });

    // Process new detections
    detections.forEach(det => {
        let bestMatch = null;
        let bestIOU = 0.3; // Minimum IOU threshold

        // Find best matching existing object
        tracked.forEach(obj => {
            if (obj.class !== det.class) return; // Only match same class

            const iou = calculateIOU(obj, det);
            if (iou > bestIOU) {
                bestIOU = iou;
                bestMatch = obj;
            }
        });

        if (bestMatch) {
            // Update existing object
            bestMatch.x = det.x;
            bestMatch.y = det.y;
            bestMatch.width = det.width;
            bestMatch.height = det.height;
            bestMatch.score = det.score;
            bestMatch.lastSeen = currentTime;
            bestMatch.matched = true;
            bestMatch.age = 0;
        } else {
            // Create new tracked object
            const newObj = {
                ...det,
                id: `${cameraIndex}-${det.class}-${currentTime}-${Math.random().toString(36).substr(2, 5)}`,
                lastSeen: currentTime,
                matched: true,
                age: 0,
                firstSeen: currentTime
            };
            tracked.push(newObj);

            // Log new object detection
            logDetection(cameraIndex, newObj, true);
        }
    });

    // Remove objects that haven't been matched for a while
    cameraSystem.trackedObjects[cameraIndex] = tracked.filter(obj => {
        if (!obj.matched && obj.age >= 2) {
            logDetection(cameraIndex, obj, false);
            return false;
        }
        return true;
    });

    // Update total objects count
    updateTotalObjectsCount();
}

// Log detection events
function logDetection(cameraIndex, obj, isNew) {
    const timestamp = formatTimestamp();
    const logElement = document.getElementById('tracking-log');
    const entry = document.createElement('div');
    entry.className = 'log-entry';

    const classColor = obj.class === 'person' ? 'log-person' : 'log-car';

    if (isNew) {
        entry.innerHTML = `
            <span class="log-timestamp">${timestamp}</span>
            <span class="log-camera">Cam ${cameraIndex+1}</span>
            <span class="${classColor}">NEW ${obj.class.toUpperCase()}</span>
            detected (ID: ${obj.id.split('-').pop()}, ${(obj.score * 100).toFixed(1)}%)
        `;
    } else {
        entry.innerHTML = `
            <span class="log-timestamp">${timestamp}</span>
            <span class="log-camera">Cam ${cameraIndex+1}</span>
            <span class="${classColor}">LOST ${obj.class.toUpperCase()}</span>
            (ID: ${obj.id.split('-').pop()}) after ${(obj.age).toFixed(1)}s
        `;
    }

    logElement.prepend(entry); // Add new entries at the top
    logElement.scrollTop = 0;

    // Keep log from growing too large
    if (logElement.children.length > 100) {
        logElement.removeChild(logElement.lastChild);
    }

    // Store in history
    cameraSystem.detectionHistory.push({
        timestamp: new Date().toISOString(),
        camera: cameraIndex,
        object: obj,
        type: isNew ? 'new' : 'lost'
    });
}

// Update total objects count display
function updateTotalObjectsCount() {
    let total = 0;
    cameraSystem.trackedObjects.forEach(cam => {
        total += cam.filter(obj => obj.matched).length;
    });
    document.getElementById('total-objects').textContent = total;
}

// Draw bounding boxes and tracking info
function drawDetections(cameraIndex) {
    const canvas = document.getElementById(`canvas${cameraIndex+1}`);
    const video = document.getElementById(`video${cameraIndex+1}`);
    const ctx = canvas.getContext('2d');

    // Set canvas dimensions to match video
    canvas.width = video.videoWidth;
    canvas.height = video.videoHeight;

    // Clear canvas
    ctx.clearRect(0, 0, canvas.width, canvas.height);

    // Count objects by class
    const counts = { person: 0, car: 0 };

    // Draw each tracked object
    cameraSystem.trackedObjects[cameraIndex].forEach(obj => {
        if (obj.class === 'person' || obj.class === 'car') {
            counts[obj.class]++;

            const color = getObjectColor(obj.id);
            const className = obj.class === 'person' ? 'person-box' : 'car-box';

            // Draw bounding box
            ctx.strokeStyle = color;
            ctx.lineWidth = 2;
            ctx.strokeRect(obj.x, obj.y, obj.width, obj.height);

            // Draw label background
            ctx.fillStyle = 'rgba(0, 0, 0, 0.5)';
            ctx.fillRect(obj.x, obj.y - 20, ctx.measureText(`${obj.class} ${(obj.score * 100).toFixed(1)}%`).width + 10, 20);

            // Draw label text
            ctx.fillStyle = color;
            ctx.font = '12px Arial';
            ctx.fillText(`${obj.class} ${(obj.score * 100).toFixed(1)}%`, obj.x + 5, obj.y - 5);

            // Draw ID
            ctx.fillStyle = 'white';
            ctx.font = '10px Arial';
            ctx.fillText(`ID: ${obj.id.split('-').pop()}`, obj.x + 5, obj.y + obj.height - 5);

            // Draw tracking duration
            const duration = ((Date.now() - obj.firstSeen) / 1000).toFixed(1);
            ctx.fillText(`${duration}s`, obj.x + obj.width - 30, obj.y + obj.height - 5);
        }
    });

    // Update object count display
    const countElement = document.querySelector(`#cam${cameraIndex+1} .object-count`);
    countElement.textContent = `Persons: ${counts.person} | Cars: ${counts.car}`;
}

// Process video frames for object detection
async function detectObjects(cameraIndex) {
    if (!cameraSystem.trackingActive) return;

    const video = document.getElementById(`video${cameraIndex+1}`);
    const canvas = document.getElementById(`canvas${cameraIndex+1}`);

    if (video.readyState < 2 || !cameraSystem.model) return;

    try {
        //@ts-ignore
        const predictions = await cameraSystem.model.detect(video);

        // Filter for persons and cars only
        const filtered = predictions.filter(p =>
            p.class === 'person' || p.class === 'car'
        ).map(p => ({
            x: p.bbox[0],
            y: p.bbox[1],
            width: p.bbox[2],
            height: p.bbox[3],
            class: p.class,
            score: p.score
        }));

        // Update tracked objects
        updateTrackedObjects(cameraIndex, filtered);

        // Draw detections
        drawDetections(cameraIndex);

    } catch (err) {
        console.error(`Detection error for camera ${cameraIndex}:`, err);
    }

    // Continue processing
    requestAnimationFrame(() => detectObjects(cameraIndex));
}

async function loadModel() {
    const status = document.getElementById('status');

    if (cameraSystem.model) return true; // Model already loaded

    try {
        cameraSystem.modelLoading = true;
        status.innerHTML = '<div class="loading-spinner"></div> Loading object tracking model...';

        // Check if TensorFlow.js and COCO-SSD are available
        //@ts-ignore
        if (typeof tf === 'undefined' || typeof cocoSsd === 'undefined') {
            // Load TensorFlow.js
            await new Promise((resolve) => {
                const script = document.createElement('script');
                script.src = "https://cdn.jsdelivr.net/npm/@tensorflow/tfjs";
                document.head.appendChild(script);
                script.onload = resolve;
            });

            // Load COCO-SSD
            await new Promise((resolve) => {
                const script = document.createElement('script');
                script.src = "https://cdn.jsdelivr.net/npm/@tensorflow-models/coco-ssd";
                document.head.appendChild(script);
                script.onload = resolve;
            });
        }

        //@ts-ignore
        cameraSystem.model = await cocoSsd.load();
        cameraSystem.modelLoading = false;
        status.textContent = "✅ Object tracking is ready";
        return true;
    } catch (err) {
        console.error("Model loading error:", err);
        status.textContent = "❌ Failed to load object detection model";
        cameraSystem.modelLoading = false;
        return false;
    }
}

async function setupCameras() {
    const status = document.getElementById('status');
    status.textContent = "⌛ Detecting cameras...";
    status.style.color = "#666";

    // Clear previous streams
    cameraSystem.streams.forEach(stream => {
        stream.getTracks().forEach(track => track.stop());
    });
    cameraSystem.streams = [];
    cameraSystem.trackedObjects = [[], []];
    cameraSystem.detectionHistory = [];
    document.getElementById('tracking-log').innerHTML = '';
    document.getElementById('total-objects').textContent = '0';

    try {
        // First get basic access
        await navigator.mediaDevices.getUserMedia({ video: true });

        // Enumerate devices with proper labels
        const devices = await navigator.mediaDevices.enumerateDevices();
        cameraSystem.devices = devices.filter(d => d.kind === 'videoinput');

        if (cameraSystem.devices.length < 2) {
            throw new Error(`Only ${cameraSystem.devices.length} cameras detected (need at least 2)`);
        }

        // Initialize each camera
        await initCamera(0, {
            width: { ideal: 1280 },
            height: { ideal: 720 },
            facingMode: 'environment'
        });
        await new Promise(resolve => setTimeout(resolve, 500));

        await initCamera(1, {
            width: { ideal: 640 },
            height: { ideal: 480 },
            facingMode: 'user'
        });

        // Verify all streams are active
        const activeStreams = cameraSystem.streams.filter(s => s.active);
        if (activeStreams.length === 2) {
            status.textContent = "✅ All cameras working";
            status.style.color = "#4CAF50";
        } else {
            throw new Error(`${activeStreams.length}/2 cameras initialized`);
        }

    } catch (err) {
        status.textContent = "❌ Error: " + err.message;
        status.style.color = "red";
    }
}

async function initCamera(index, customConstraints = {}) {
    const video = document.getElementById(`video${index+1}`);
    const camElement = document.getElementById(`cam${index+1}`);
    const label = camElement.querySelector('.camera-label');

    try {
        const device = cameraSystem.devices[index];

        const constraints = {
            video: {
                deviceId: device.deviceId ? { exact: device.deviceId } : undefined,
                ...customConstraints
            }
        };

        const stream = await navigator.mediaDevices.getUserMedia(constraints);

        // Verify this is a new stream
        if (cameraSystem.streams.some(s =>
            s.getVideoTracks()[0].getSettings().deviceId ===
            stream.getVideoTracks()[0].getSettings().deviceId
        )) {
            throw new Error("Duplicate camera stream detected");
        }

        video.srcObject = stream;
        cameraSystem.streams.push(stream);

        camElement.style.border = "2px solid #4CAF50";
        label.textContent = `Cam ${index+1}: ${device.label || 'Device '+index}`;

    } catch (err) {
        console.error(`Camera ${index+1} error:`, err);
        camElement.innerHTML = `
            <div class="camera-error">
                ${err.message.includes("Duplicate") ?
                 'Identical Camera Conflict' :
                 `Camera ${index+1} Error`}
            </div>
        `;
        camElement.style.border = "2px solid #F44336";
        throw err;
    }
}

// Start tracking function
async function startTracking() {
    const status = document.getElementById('status');
    const startBtn = document.getElementById('start-tracking-btn');
    const stopBtn = document.getElementById('stop-tracking-btn');

    // Disable start button during loading
    startBtn.disabled = true;
    startBtn.textContent = "Loading...";
    startBtn.style.background = "#9E9E9E";

    // Show loading status
    status.innerHTML = '<div class="loading-spinner"></div> Loading object detection model...';

    try {
        // Load model if not already loaded
        const modelLoaded = await loadModel();

        if (!modelLoaded) {
            status.textContent = "❌ Failed to load object detection model";
            startBtn.disabled = false;
            startBtn.textContent = "Start Tracking";
            startBtn.style.background = "#4CAF50";
            return;
        }

        // Start tracking
        cameraSystem.trackingActive = true;
        status.textContent = "🔍 Tracking persons and cars...";

        // Start detection for each camera
        for (let i = 0; i < cameraSystem.streams.length; i++) {
            detectObjects(i);
        }

        // Toggle buttons
        startBtn.style.display = "none";
        stopBtn.style.display = "inline-block";

    } catch (err) {
        console.error("Tracking start error:", err);
        status.textContent = "❌ Error starting tracking: " + err.message;
        startBtn.disabled = false;
        startBtn.textContent = "Start Tracking";
        startBtn.style.background = "#4CAF50";
    }
}

// Stop tracking function
function stopTracking() {
    cameraSystem.trackingActive = false;
    document.getElementById('status').textContent = "✅ Tracking stopped";

    // Toggle buttons
    document.getElementById('start-tracking-btn').style.display = "inline-block";
    document.getElementById('stop-tracking-btn').style.display = "none";

    // Reset start button
    const startBtn = document.getElementById('start-tracking-btn');
    startBtn.disabled = false;
    startBtn.textContent = "Start Tracking";
    startBtn.style.background = "#4CAF50";
}

// Clear log function
function clearLog() {
    document.getElementById('tracking-log').innerHTML = '';
    cameraSystem.detectionHistory = [];
}

// Start initialization
setTimeout(async () => {
    await setupCameras();
}, 500);

// Event listeners
document.getElementById('retry-btn').addEventListener('click', setupCameras);
document.getElementById('start-tracking-btn').addEventListener('click', startTracking);
document.getElementById('stop-tracking-btn').addEventListener('click', stopTracking);
document.getElementById('clear-log-btn').addEventListener('click', clearLog);
</script>
"""))

Modified Version to determine track Persons or cars

In [118]:
from IPython.display import display, HTML

display(HTML("""
<div style="margin: 20px; text-align: center;">
    <h2>Multi Objects Tracking using Multi Cameras</h2>
    <!-- Tracking Mode Selection -->
    <div style="margin: 15px 0;">
        <label for="tracking-mode" style="margin-right: 10px;">Tracking Mode:</label>
        <select id="tracking-mode" style="padding: 5px 10px; border-radius: 4px;">
            <option value="both">Both Persons and Cars</option>
            <option value="person">Only Persons</option>
            <option value="car">Only Cars</option>
        </select>
    </div>
    <div id="camera-container" style="display: flex; justify-content: center; gap: 20px; flex-wrap: wrap;">
        <!-- Camera 1 -->
        <div id="cam1" style="width: 320px; height: 240px; border: 2px solid #ddd; background: #000; overflow: hidden; position: relative;">
            <video id="video1" autoplay playsinline muted style="width: 100%; height: 100%; object-fit: cover;"></video>
            <canvas id="canvas1" style="position: absolute; top: 0; left: 0; width: 100%; height: 100%;"></canvas>
            <div class="camera-label">Camera 1</div>
            <div class="object-count" style="position: absolute; bottom: 10px; left: 10px; background: rgba(0,0,0,0.7); color: white; padding: 2px 8px; border-radius: 4px;">
                Persons: 0 | Cars: 0
            </div>
        </div>
        <!-- Camera 2 -->
        <div id="cam2" style="width: 320px; height: 240px; border: 2px solid #ddd; background: #000; overflow: hidden; position: relative;">
            <video id="video2" autoplay playsinline muted style="width: 100%; height: 100%; object-fit: cover;"></video>
            <canvas id="canvas2" style="position: absolute; top: 0; left: 0; width: 100%; height: 100%;"></canvas>
            <div class="camera-label">Camera 2</div>
            <div class="object-count" style="position: absolute; bottom: 10px; left: 10px; background: rgba(0,0,0,0.7); color: white; padding: 2px 8px; border-radius: 4px;">
                Persons: 0 | Cars: 0
            </div>
        </div>
    </div>
    <div id="tracking-info" style="margin-top: 20px; padding: 15px; background: #f5f5f5; border-radius: 5px; text-align: left;">
        <div style="display: flex; justify-content: space-between; align-items: center;">
            <h3 style="margin-top: 0;">Tracking Information</h3>
            <button id="clear-log-btn" style="padding: 5px 10px; background: #f44336; color: white; border: none; border-radius: 3px;">Clear Log</button>
        </div>
        <div id="tracking-log" style="max-height: 200px; overflow-y: auto; font-family: monospace; background: white; padding: 10px; border: 1px solid #ddd; border-radius: 4px;"></div>
        <div style="margin-top: 10px; display: flex; gap: 10px;">
            <div style="flex: 1; background: #FF5722; color: white; padding: 5px; border-radius: 3px;">Person</div>
            <div style="flex: 1; background: #3F51B5; color: white; padding: 5px; border-radius: 3px;">Car</div>
            <div style="flex: 2; background: #4CAF50; color: white; padding: 5px; border-radius: 3px;">Active Objects: <span id="total-objects">0</span></div>
        </div>
    </div>
    <p id="status" style="color: #666; margin-top: 20px;">Initializing cameras...</p>
    <div>
        <button id="retry-btn" style="padding: 10px 20px; background: #2196F3; color: white; border: none; border-radius: 5px; margin-top: 15px;">Reinitialize Cameras</button>
        <button id="start-tracking-btn" style="padding: 10px 20px; background: #4CAF50; color: white; border: none; border-radius: 5px; margin-top: 15px; margin-left: 10px;">Start Tracking</button>
        <button id="stop-tracking-btn" style="padding: 10px 20px; background: #F44336; color: white; border: none; border-radius: 5px; margin-top: 15px; margin-left: 10px; display: none;">Stop Tracking</button>
    </div>
</div>

<style>
.camera-label {
    position: absolute;
    top: 10px;
    left: 10px;
    background: rgba(0,0,0,0.7);
    color: white;
    padding: 2px 8px;
    border-radius: 4px;
}
.camera-error {
    width:100%;
    height:100%;
    display:flex;
    align-items:center;
    justify-content:center;
    background:#222;
    color:white;
}
.tracking-box {
    position: absolute;
    border: 2px solid;
    font-size: 10px;
    font-weight: bold;
    padding: 1px;
    background: rgba(0,0,0,0.3);
}
.person-box {
    border-color: #FF5722;
    color: #FF5722;
}
.car-box {
    border-color: #3F51B5;
    color: #3F51B5;
}
.loading-spinner {
    border: 4px solid #f3f3f3;
    border-top: 4px solid #3498db;
    border-radius: 50%;
    width: 30px;
    height: 30px;
    animation: spin 2s linear infinite;
    margin: 20px auto;
}
@keyframes spin {
    0% { transform: rotate(0deg); }
    100% { transform: rotate(360deg); }
}
.log-entry {
    margin: 3px 0;
    padding: 3px;
    border-bottom: 1px solid #eee;
}
.log-timestamp {
    color: #666;
    margin-right: 8px;
}
.log-camera {
    font-weight: bold;
    margin-right: 8px;
}
.log-person {
    color: #FF5722;
}
.log-car {
    color: #3F51B5;
}
</style>

<script>
const cameraSystem = {
    streams: [],
    devices: [],
    model: null,
    trackingActive: false,
    modelLoading: false,
    trackedObjects: [[], []],
    objectColors: {},
    detectionHistory: [],
    trackingMode: 'both' // Default mode
};

function setTrackingMode(mode) {
    cameraSystem.trackingMode = mode;
    const statusText = {
        'both': 'Tracking persons and cars',
        'person': 'Tracking only persons',
        'car': 'Tracking only cars'
    };
    document.getElementById('status').textContent = `✅ ${statusText[mode]}`;

    // Clear existing detections when mode changes
    cameraSystem.trackedObjects = [[], []];
    clearLog();
}
//------------------------------------------------------------------------------
// Format timestamp for logging
function formatTimestamp() {
    const now = new Date();
    return now.toLocaleTimeString() + '.' + now.getMilliseconds().toString().padStart(3, '0');
}

// Generate a random color for each tracked object
function getObjectColor(trackId) {
    if (!cameraSystem.objectColors[trackId]) {
        const hue = Math.floor(Math.random() * 360);
        cameraSystem.objectColors[trackId] = `hsl(${hue}, 100%, 50%)`;
    }
    return cameraSystem.objectColors[trackId];
}

// Calculate Intersection over Union (IoU)
function calculateIOU(box1, box2) {
    const x1 = Math.max(box1.x, box2.x);
    const y1 = Math.max(box1.y, box2.y);
    const x2 = Math.min(box1.x + box1.width, box2.x + box2.width);
    const y2 = Math.min(box1.y + box1.height, box2.y + box2.height);

    const intersection = Math.max(0, x2 - x1) * Math.max(0, y2 - y1);
    const area1 = box1.width * box1.height;
    const area2 = box2.width * box2.height;

    return intersection / (area1 + area2 - intersection);
}

// Track objects between frames
function updateTrackedObjects(cameraIndex, detections) {
    const currentTime = Date.now();
    const tracked = cameraSystem.trackedObjects[cameraIndex];

    // Mark all current tracked objects as unmatched
    tracked.forEach(obj => {
        obj.matched = false;
        obj.age = (currentTime - obj.lastSeen) / 1000; // Age in seconds
    });

    // Process new detections
    detections.forEach(det => {
        let bestMatch = null;
        let bestIOU = 0.3; // Minimum IOU threshold

        // Find best matching existing object
        tracked.forEach(obj => {
            if (obj.class !== det.class) return; // Only match same class

            const iou = calculateIOU(obj, det);
            if (iou > bestIOU) {
                bestIOU = iou;
                bestMatch = obj;
            }
        });

        if (bestMatch) {
            // Update existing object
            bestMatch.x = det.x;
            bestMatch.y = det.y;
            bestMatch.width = det.width;
            bestMatch.height = det.height;
            bestMatch.score = det.score;
            bestMatch.lastSeen = currentTime;
            bestMatch.matched = true;
            bestMatch.age = 0;
        } else {
            // Create new tracked object
            const newObj = {
                ...det,
                id: `${cameraIndex}-${det.class}-${currentTime}-${Math.random().toString(36).substr(2, 5)}`,
                lastSeen: currentTime,
                matched: true,
                age: 0,
                firstSeen: currentTime
            };
            tracked.push(newObj);

            // Log new object detection
            logDetection(cameraIndex, newObj, true);
        }
    });

    // Remove objects that haven't been matched for a while
    cameraSystem.trackedObjects[cameraIndex] = tracked.filter(obj => {
        if (!obj.matched && obj.age >= 2) {
            logDetection(cameraIndex, obj, false);
            return false;
        }
        return true;
    });

    // Update total objects count
    updateTotalObjectsCount();
}

// Log detection events
function logDetection(cameraIndex, obj, isNew) {
    const timestamp = formatTimestamp();
    const logElement = document.getElementById('tracking-log');
    const entry = document.createElement('div');
    entry.className = 'log-entry';

    const classColor = obj.class === 'person' ? 'log-person' : 'log-car';

    if (isNew) {
        entry.innerHTML = `
            <span class="log-timestamp">${timestamp}</span>
            <span class="log-camera">Cam ${cameraIndex+1}</span>
            <span class="${classColor}">NEW ${obj.class.toUpperCase()}</span>
            detected (ID: ${obj.id.split('-').pop()}, ${(obj.score * 100).toFixed(1)}%)
        `;
    } else {
        entry.innerHTML = `
            <span class="log-timestamp">${timestamp}</span>
            <span class="log-camera">Cam ${cameraIndex+1}</span>
            <span class="${classColor}">LOST ${obj.class.toUpperCase()}</span>
            (ID: ${obj.id.split('-').pop()}) after ${(obj.age).toFixed(1)}s
        `;
    }

    logElement.prepend(entry); // Add new entries at the top
    logElement.scrollTop = 0;

    // Keep log from growing too large
    if (logElement.children.length > 100) {
        logElement.removeChild(logElement.lastChild);
    }

    // Store in history
    cameraSystem.detectionHistory.push({
        timestamp: new Date().toISOString(),
        camera: cameraIndex,
        object: obj,
        type: isNew ? 'new' : 'lost'
    });
}

// Update total objects count display
function updateTotalObjectsCount() {
    let total = 0;
    cameraSystem.trackedObjects.forEach(cam => {
        total += cam.filter(obj => obj.matched).length;
    });
    document.getElementById('total-objects').textContent = total;
}
//------------------------------------------------------------------------------
// Draw bounding boxes and tracking info
function drawDetections(cameraIndex) {
    const canvas = document.getElementById(`canvas${cameraIndex+1}`);
    const video = document.getElementById(`video${cameraIndex+1}`);
    const ctx = canvas.getContext('2d');

    // Set canvas dimensions to match video
    canvas.width = video.videoWidth;
    canvas.height = video.videoHeight;

    // Clear canvas
    ctx.clearRect(0, 0, canvas.width, canvas.height);

    // Count objects by class
    const counts = { person: 0, car: 0 };

    // Draw each tracked object
    cameraSystem.trackedObjects[cameraIndex].forEach(obj => {
        if (obj.class === 'person' || obj.class === 'car') {
            counts[obj.class]++;

            const color = getObjectColor(obj.id);
            const className = obj.class === 'person' ? 'person-box' : 'car-box';
            const classLabel = obj.class.toUpperCase(); // Convert to uppercase for better visibility

            // Draw bounding box
            ctx.strokeStyle = color;
            ctx.lineWidth = 2;
            ctx.strokeRect(obj.x, obj.y, obj.width, obj.height);

            // Draw label background (larger to accommodate class name)
            const labelText = `${classLabel} ${(obj.score * 100).toFixed(1)}%`;
            const labelWidth = ctx.measureText(labelText).width + 10;
            ctx.fillStyle = 'rgba(0, 0, 0, 0.5)';
            ctx.fillRect(obj.x, obj.y - 20, labelWidth, 20);

            // Draw class name and confidence
            ctx.fillStyle = color;
            ctx.font = 'bold 12px Arial'; // Make font bold for better visibility
            ctx.fillText(labelText, obj.x + 5, obj.y - 5);

            // Draw ID
            ctx.fillStyle = 'white';
            ctx.font = '10px Arial';
            ctx.fillText(`ID: ${obj.id.split('-').pop()}`, obj.x + 5, obj.y + obj.height - 5);

            // Draw tracking duration
            const duration = ((Date.now() - obj.firstSeen) / 1000).toFixed(1);
            ctx.fillText(`${duration}s`, obj.x + obj.width - 30, obj.y + obj.height - 5);
        }
    });

    // Update object count display
    const countElement = document.querySelector(`#cam${cameraIndex+1} .object-count`);
    if (cameraSystem.trackingMode === 'person') {
        countElement.textContent = `Persons: ${counts.person}`;
    } else if (cameraSystem.trackingMode === 'car') {
        countElement.textContent = `Cars: ${counts.car}`;
    } else {
        countElement.textContent = `Persons: ${counts.person} | Cars: ${counts.car}`;
    }
}
//-----------------------------------------------------------------------------
//--- Process video frames for object detection and tracking -------

async function detectObjects(cameraIndex) {
    if (!cameraSystem.trackingActive) return;

    const video = document.getElementById(`video${cameraIndex+1}`);
    const canvas = document.getElementById(`canvas${cameraIndex+1}`);

    if (video.readyState < 2 || !cameraSystem.model) return;

    try {

        const predictions = await cameraSystem.model.detect(video);

        let filtered = [];
        if (cameraSystem.trackingMode === 'person') {
            filtered = predictions.filter(p => p.class === 'person');
        } else if (cameraSystem.trackingMode === 'car') {
            filtered = predictions.filter(p => p.class === 'car');
        } else { // 'both'
            filtered = predictions.filter(p => p.class === 'person' || p.class === 'car');
        }

        filtered = filtered.map(p => ({
            x: p.bbox[0],
            y: p.bbox[1],
            width: p.bbox[2],
            height: p.bbox[3],
            class: p.class,
            score: p.score
        }));

        // Update tracked objects
        updateTrackedObjects(cameraIndex, filtered);

        // Draw detections
        drawDetections(cameraIndex);

    } catch (err) {
        console.error(`Detection error for camera ${cameraIndex}:`, err);
    }

    // Continue processing
    requestAnimationFrame(() => detectObjects(cameraIndex));
}
//-----------------------------------------------------------------------------
async function loadModel() {
    const status = document.getElementById('status');

    if (cameraSystem.model) return true; // Model already loaded

    try {
        cameraSystem.modelLoading = true;
        status.innerHTML = '<div class="loading-spinner"></div> Loading object tracking model...';

        // Check if TensorFlow.js and developed model is ready
        //@ts-ignore
        if (typeof tf === 'undefined' || typeof cocoSsd === 'undefined') {
            // Load TensorFlow.js
            await new Promise((resolve) => {
                const script = document.createElement('script');
                script.src = "https://cdn.jsdelivr.net/npm/@tensorflow/tfjs";
                document.head.appendChild(script);
                script.onload = resolve;
            });

            // Load developed model
            await new Promise((resolve) => {
                const script = document.createElement('script');
                script.src = "https://cdn.jsdelivr.net/npm/@tensorflow-models/coco-ssd";
                document.head.appendChild(script);
                script.onload = resolve;
            });
        }

        //@ts-ignore
        cameraSystem.model = await cocoSsd.load();
        cameraSystem.modelLoading = false;
        status.textContent = "✅ Object tracking is ready";
        return true;
    } catch (err) {
        console.error("Model loading error:", err);
        status.textContent = "❌ Failed to load object detection model";
        cameraSystem.modelLoading = false;
        return false;
    }
}
//------------------------------------------------------------------------------
async function setupCameras() {
    const status = document.getElementById('status');
    status.textContent = "⌛ Detecting cameras...";
    status.style.color = "#666";

    // Clear previous streams
    cameraSystem.streams.forEach(stream => {
        stream.getTracks().forEach(track => track.stop());
    });
    cameraSystem.streams = [];
    cameraSystem.trackedObjects = [[], []];
    cameraSystem.detectionHistory = [];
    document.getElementById('tracking-log').innerHTML = '';
    document.getElementById('total-objects').textContent = '0';

    try {
        // First get basic access
        await navigator.mediaDevices.getUserMedia({ video: true });

        // Enumerate devices with proper labels
        const devices = await navigator.mediaDevices.enumerateDevices();
        cameraSystem.devices = devices.filter(d => d.kind === 'videoinput');

        if (cameraSystem.devices.length < 2) {
            throw new Error(`Only ${cameraSystem.devices.length} cameras detected (need at least 2)`);
        }

        // Initialize each camera
        await initCamera(0, {
            width: { ideal: 1280 },
            height: { ideal: 720 },
            facingMode: 'environment'
        });
        await new Promise(resolve => setTimeout(resolve, 500));

        await initCamera(1, {
            width: { ideal: 640 },
            height: { ideal: 480 },
            facingMode: 'user'
        });

        // Verify all streams are active
        const activeStreams = cameraSystem.streams.filter(s => s.active);
        if (activeStreams.length === 2) {
            status.textContent = "✅ All cameras working";
            status.style.color = "#4CAF50";
        } else {
            throw new Error(`${activeStreams.length}/2 cameras initialized`);
        }

    } catch (err) {
        status.textContent = "❌ Error: " + err.message;
        status.style.color = "red";
    }
}

async function initCamera(index, customConstraints = {}) {
    const video = document.getElementById(`video${index+1}`);
    const camElement = document.getElementById(`cam${index+1}`);
    const label = camElement.querySelector('.camera-label');

    try {
        const device = cameraSystem.devices[index];

        const constraints = {
            video: {
                deviceId: device.deviceId ? { exact: device.deviceId } : undefined,
                ...customConstraints
            }
        };

        const stream = await navigator.mediaDevices.getUserMedia(constraints);

        // Verify this is a new stream
        if (cameraSystem.streams.some(s =>
            s.getVideoTracks()[0].getSettings().deviceId ===
            stream.getVideoTracks()[0].getSettings().deviceId
        )) {
            throw new Error("Duplicate camera stream detected");
        }

        video.srcObject = stream;
        cameraSystem.streams.push(stream);

        camElement.style.border = "2px solid #4CAF50";
        label.textContent = `Cam ${index+1}: ${device.label || 'Device '+index}`;

    } catch (err) {
        console.error(`Camera ${index+1} error:`, err);
        camElement.innerHTML = `
            <div class="camera-error">
                ${err.message.includes("Duplicate") ?
                 'Identical Camera Conflict' :
                 `Camera ${index+1} Error`}
            </div>
        `;
        camElement.style.border = "2px solid #F44336";
        throw err;
    }
}

// Start tracking function
async function startTracking() {
    const status = document.getElementById('status');
    const startBtn = document.getElementById('start-tracking-btn');
    const stopBtn = document.getElementById('stop-tracking-btn');

    // Disable start button during loading
    startBtn.disabled = true;
    startBtn.textContent = "Loading...";
    startBtn.style.background = "#9E9E9E";

    // Show loading status
    status.innerHTML = '<div class="loading-spinner"></div> Loading object detection model...';

    try {
        // Load model if not already loaded
        const modelLoaded = await loadModel();

        if (!modelLoaded) {
            status.textContent = "❌ Failed to load object detection model";
            startBtn.disabled = false;
            startBtn.textContent = "Start Tracking";
            startBtn.style.background = "#4CAF50";
            return;
        }

        // Start tracking
        cameraSystem.trackingActive = true;
        status.textContent = "🔍 Tracking persons and cars...";

        // Start detection for each camera
        for (let i = 0; i < cameraSystem.streams.length; i++) {
            detectObjects(i);
        }

        // Toggle buttons
        startBtn.style.display = "none";
        stopBtn.style.display = "inline-block";

    } catch (err) {
        console.error("Tracking start error:", err);
        status.textContent = "❌ Error starting tracking: " + err.message;
        startBtn.disabled = false;
        startBtn.textContent = "Start Tracking";
        startBtn.style.background = "#4CAF50";
    }
}

// Stop tracking function
function stopTracking() {
    cameraSystem.trackingActive = false;
    document.getElementById('status').textContent = "✅ Tracking stopped";

    // Toggle buttons
    document.getElementById('start-tracking-btn').style.display = "inline-block";
    document.getElementById('stop-tracking-btn').style.display = "none";

    // Reset start button
    const startBtn = document.getElementById('start-tracking-btn');
    startBtn.disabled = false;
    startBtn.textContent = "Start Tracking";
    startBtn.style.background = "#4CAF50";
}

// Clear log function
function clearLog() {
    document.getElementById('tracking-log').innerHTML = '';
    cameraSystem.detectionHistory = [];
}

// Start initialization
setTimeout(async () => {
    await setupCameras();
}, 500);

// Event listeners
document.getElementById('retry-btn').addEventListener('click', setupCameras);
document.getElementById('start-tracking-btn').addEventListener('click', startTracking);
document.getElementById('stop-tracking-btn').addEventListener('click', stopTracking);
document.getElementById('clear-log-btn').addEventListener('click', clearLog);
document.getElementById('tracking-mode').addEventListener('change', function() {
    setTrackingMode(this.value);
    if (cameraSystem.trackingActive) {
        // Restart tracking if it was active
        stopTracking();
        startTracking();
    }
});
</script>
"""))